In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedKFold


def crossCheck(cross,foldnum):
    scores=[]
    for i in range(len(cross)):

        ctrain=[]
        ctrain_y=[]
        testnum=foldnum-i-1
        testme=[]
        testme_y=[]
        
        testme=np.array(testme)
        testme_y=np.array(testme_y)
        j=0
        while j<len(cross):

            
            if j!=testnum:
                for thing in cross[j]:
                    ctrain.append(thing[0])
                    ctrain_y.append(thing[1])
               
            j+=1
            
###  PREPARING DATA  ###
        
#Reading in data from files
meds=open('meds.txt').readline().strip().split('\t')
data=pd.read_csv('diabetic_data.csv')

#Diagnosis Features
diag=data[['diag_1','diag_2','diag_3']]

#dropping incomplete and unneeded data
data=data.drop(['admission_type_id','discharge_disposition_id','admission_source_id','time_in_hospital','max_glu_serum','number_inpatient','number_emergency','number_outpatient','num_procedures','num_lab_procedures','encounter_id','patient_nbr','weight','payer_code','A1Cresult','medical_specialty','max_glu_serum','diag_1','diag_2','diag_3'],axis=1)
data=data.drop(data[data.race=='?'].index)
data=data.drop(data[data.gender=='Unknown/Invalid'].index)

#Readmission Features
read=data['readmitted']
read=read.str.upper()
read=read.replace(['NO','<30','>30'],[0,1,2])

#Giving Categorical Features Numerical Values
data=data.replace(['None','Female'],0)
data=data.replace(['Male'],1)
#data=data.replace(['[0-10)','[10-20)','[20-30)','[30-40)','[40-50)','[50-60)','[60-70)','[70-80)','[80-90)','[90-100)'],[0,1,2,3,4,5,6,7,8,9])
data=data.replace(['Down'],2)
#data=data.replace(['Other','Hispanic'],[3,4])
r=pd.get_dummies(data[['race','diabetesMed']])
change=pd.get_dummies(data['change'])
a=pd.get_dummies(data['age'])
meddum=pd.get_dummies(data[meds])

#Recompiling dataset
newdata=pd.concat([data,r,a,meddum,change],axis=1)
newdata=newdata.drop(['age','race','readmitted','change','diabetesMed'],axis=1)
newdata=newdata.drop(meds,axis=1)

#normalization
transformer = Normalizer().fit(newdata)
tran=transformer.transform(newdata)
print(tran)

#converting to numpy
read=np.array(read)

#splitting between training and testing
train_X, test_X, train_y, test_y = train_test_split(tran, read, test_size=0.3,random_state=0)



[[0.         0.18569534 0.18569534 ... 0.         0.         0.18569534]
 [0.         0.8660254  0.4330127  ... 0.         0.04811252 0.        ]
 [0.         0.85349181 0.3939193  ... 0.         0.         0.06565322]
 ...
 [0.05997601 0.53978413 0.77968819 ... 0.         0.05997601 0.        ]
 [0.         0.89625816 0.38411064 ... 0.         0.04267896 0.        ]
 [0.09205746 0.27617239 0.82851716 ... 0.         0.         0.09205746]]


In [ ]:
###  MODEL S
model = LogisticRegression()
rfe=RFE(estimator=model, n_features_to_select=7, step=1)
rfe.fit(train_X,train_y)
#print(rfe.get_params)
# hi=list(newdata.columns.values)
# hey=list(rfe.ranking_)
# i=0
# print(hi)
# for thing in hey:
#     if thing ==1:
#         print(hi[i])
#     i+=1


In [ ]:
model = SVC(kernel='linear')

model.fit(train_X,train_y)

pred=(model.predict(test_X))
testy=np.array(test_y)
print(len(pred))
i=0
falsep=0
falsen=0
wrongn=0
rightp=0
# for thing in pred:
#     if thing != testy[i] and thing==1:
#         #wrong+=1
#         falsep+=1
#         #print(thing,testy[i])
#     elif thing!=testy[i] and thing==0:
#         falsen+=1
#     elif thing==testy[i] and thing==1:
#         rightp+=1
#     else: 
#         wrongn+=1
    #right+=1
    
    
    i+=1
print(rightp,wrongn,falsep,falsen)
p=rightp/(rightp+falsep)
r=rightp/(falsen+rightp)
f=2*((p*r)/(p+r))
print(p,r,f)
print(accuracy_score(pred,test_y))


In [ ]:
from sklearn.metrics import f1_score
print(f1_score(pred,test_y))